In [1]:
import pandas as pd
import numpy as np
import torch
import scripts
from functools import lru_cache
import torchmetrics
from torch import nn
import optuna

/home/karolyi/miniconda/envs/project_thesis/lib/python3.13/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data loading

First we load the data. The basic idea is to create dictionaries with features associated to the drugs and cell-lines. In principle, the splits and the data shouldn't be changed

In [7]:
@lru_cache(maxsize = None)
def get_data(n_fold = 0, fp_radius = 2):
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()
    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    tensor_exp = torch.Tensor(filtered_rna.to_numpy())
    cell_dict = {cell: tensor_exp[i] for i, cell in enumerate(filtered_rna.index.to_numpy())}
    data = pd.read_csv("data/GDSC12.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = data.query("SANGER_MODEL_ID in @cell_dict.keys() & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()
    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    test_lines = folds[0]
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold]
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")
    return (scripts.OmicsDataset_drugwise(cell_dict, drug_dict, train_data),
    scripts.OmicsDataset_drugwise(cell_dict, drug_dict, validation_data),
    scripts.OmicsDataset_drugwise(cell_dict, drug_dict, test_data))

# Configuration

we declare the configuration, this is going to be model-specific and we get the datasets

In [8]:
config = {"features" : {"fp_radius":2},
          "optimizer": {"batch_size": 3,
                        "clip_norm":19,
                        "learning_rate":0.0004592646200179472,
                        "stopping_patience":15},
          "model":{"embed_dim":485,
                 "hidden_dim":696,
                 "dropout":0.48541242824674574,
                 "n_layers": 4,
                 "norm": "batchnorm"},
         "env": {"fold": 0,
                 "device":"cpu",
                 "max_epochs": 100,
                 "search_hyperparameters":True}}

In [9]:
train_dataset, validation_dataset, test_dataset = get_data(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"])



[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerator
[16:33:01] DEPRECATION WARNING: please use MorganGenerat

In [7]:
cell_features, drug_features, target, drug_index, cell_indices, labels = train_dataset[4]

ins = train_dataset[7]
print(ins[5].shape)
print(ins[4].shape)

print("Drug Features Shape:", drug_features.shape)
print("Cell Features Shape:", cell_features.shape)  
print("Labels Shape:", labels.shape)
print("Target Shape:", target.shape)               
print("Drug Index Shape:", drug_index.shape)      
print("Cell Indices Shape:", cell_indices.shape) 




torch.Size([325])
torch.Size([325])
Drug Features Shape: torch.Size([323, 2048])
Cell Features Shape: torch.Size([323, 777])
Labels Shape: torch.Size([323])
Target Shape: torch.Size([323])
Drug Index Shape: torch.Size([323])
Cell Indices Shape: torch.Size([323])


# Hyperparameter optimization

we wrap the function for training the model in a function that can be used by optuna

In [10]:
def train_model_optuna(trial, config):
    def pruning_callback(epoch, train_r):
        trial.report(train_r, step = epoch)
        if np.isnan(train_r):
            raise optuna.TrialPruned()
        if trial.should_prune():
            raise optuna.TrialPruned()
    config["model"] = {"embed_dim": trial.suggest_int("embed_dim", 64, 512),
                    "hidden_dim": trial.suggest_int("hidden_dim", 64, 2048),
                    "n_layers": trial.suggest_int("n_layers", 1, 6),
                    "norm": trial.suggest_categorical("norm", ["layernorm","layernorm", None]),
                    "dropout": trial.suggest_float("dropout", 0.0, 0.5)}
    config["optimizer"] = { "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True),
                            "clip_norm": trial.suggest_int("clip_norm", 0.1, 20),
                            "batch_size": trial.suggest_int("batch_size", 2, 10),
                            "stopping_patience":10}
    try:
        R, model = scripts.train_model(config,
                                       train_dataset,
                                       validation_dataset,
                                       use_momentum=True,
                                       callback_epoch = pruning_callback)
        return R
    except Exception as e:
        print(e)
        return 0

In [11]:


if config["env"]["search_hyperparameters"]:
    study_name = f"baseline_model"
    storage_name = "sqlite:///studies/{}.db".format(study_name)
    study = optuna.create_study(study_name=study_name,
                                storage=storage_name,
                                direction='maximize',
                                load_if_exists=True,
                                pruner=optuna.pruners.MedianPruner(n_startup_trials=30,
                                                               n_warmup_steps=5,
                                                               interval_steps=5))
    objective = lambda x: train_model_optuna(x, config)
    study.optimize(objective, n_trials=40)
    best_config = study.best_params
    print(best_config)
    config["model"]["embed_dim"] = best_config["embed_dim"]
    config["model"]["hidden_dim"] = best_config["hidden_dim"]
    config["model"]["n_layers"] = best_config["n_layers"]
    config["model"]["norm"] = best_config["norm"]
    config["model"]["dropout"] = best_config["dropout"]
    config["optimizer"]["learning_rate"] = best_config["learning_rate"]
    config["optimizer"]["clip_norm"] = best_config["clip_norm"]
    config["optimizer"]["batch_size"] = best_config["batch_size"]



[I 2025-01-07 10:46:46,214] Using an existing study with name 'baseline_model' instead of creating a new one.
[W 2025-01-07 10:46:46,663] Trial 366 failed with parameters: {'embed_dim': 321, 'hidden_dim': 1160, 'n_layers': 6} because of the following error: ValueError('CategoricalDistribution does not support dynamic value space.').
Traceback (most recent call last):
  File "/home/karolyi/miniconda/envs/project_thesis/lib/python3.13/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_1817391/989827914.py", line 11, in <lambda>
    objective = lambda x: train_model_optuna(x, config)
                          ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
  File "/tmp/ipykernel_1817391/3681637092.py", line 11, in train_model_optuna
    "norm": trial.suggest_categorical("norm", ["layernorm","layernorm", None]),
            ~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/karolyi/miniconda/envs/project_th

ValueError: CategoricalDistribution does not support dynamic value space.

# Model training and evaluation

After we have a set of optimal hyperparameters we train our model. The train model function could be changed, but:
- test_dataset cannot be used until we call the final evaluation step
- the evaluation step cannot be modified, it must take the model produced by your pipeline, a dataloader that provides the correct data for your model, and the final metrics have to be printed

In [26]:
'''
def custom_collate_fn(batch):
    # Determine the maximum length in the batch
    max_len = max(item[4].size(0) for item in batch)  
    #print(max_len)

    padded_batch = []
    for cell_features, drug_features, target, drug_index, cell_indices, labels in batch:
        pad_len = max_len - cell_features.size(0)

        # Pad Cell Features to match max_len (rows only)
        padded_cell_features = torch.nn.functional.pad(cell_features, (0, 0, 0, pad_len))  # Pad rows
        padded_drug_feature = torch.nn.functional.pad(drug_features, (0, 0, 0, pad_len))

        # Pad associated vectors to match max_len
        padded_labels = torch.nn.functional.pad(labels, (0, pad_len))
        padded_target = torch.nn.functional.pad(target, (0, pad_len))
        padded_drug_index = torch.nn.functional.pad(drug_index, (0, pad_len))
        padded_cell_indices = torch.nn.functional.pad(cell_indices, (0, pad_len))

        padded_batch.append((padded_cell_features, padded_drug_feature, padded_target,
                             padded_drug_index, padded_cell_indices, padded_labels))

    # Stack all tensors
    cell_features = torch.stack([item[0] for item in padded_batch])
    drug_features = torch.stack([item[1] for item in padded_batch])
    target = torch.stack([item[2] for item in padded_batch])
    drug_index = torch.stack([item[3] for item in padded_batch])
    cell_indices = torch.stack([item[4] for item in padded_batch])
    labels = torch.stack([item[5] for item in padded_batch])

    return cell_features, drug_features, target, drug_index, cell_indices, labels

'''

def collate_fn_custom(batch):
    omics = []
    drugs = []
    targets = []
    cell_ids = []
    drug_ids = []
    labels = [] 

    for o, d, t, cid, did, r in batch:
        omics.append(o)  # Add each omics tensor
        drugs.append(d)  # Add each drugs tensor
        targets.append(t)  # Add each target tensor
        cell_ids.append(cid)  # Add each cell ID tensor
        drug_ids.append(did)  # Add each drug ID tensor
        labels.append(r)  # Add each label tensor

    # Concatenate along the first dimension for all tensors
    omics = torch.cat(omics, dim=0)
    drugs = torch.cat(drugs, dim=0)
    targets = torch.cat(targets, dim=0)
    cell_ids = torch.cat(cell_ids, dim=0)
    drug_ids = torch.cat(drug_ids, dim=0)
    labels = torch.cat(labels, dim=0)

    return omics, drugs, targets, cell_ids, drug_ids, labels


In [31]:
import scripts

%reload_ext autoreload

%autoreload 2

_, model = scripts.train_model(config, train_dataset,  validation_dataset=None, use_momentum=True, callback_epoch=None, collate_fn=collate_fn_custom)
device = torch.device(config["env"]["device"])
metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
    {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="drugs",
                          average="macro",
                          residualize=True),
    "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="cell_lines",
                          average="macro",
                          residualize=False),
    "MSE":torchmetrics.MeanSquaredError()}))
metrics.to(device)
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                      batch_size=config["optimizer"]["batch_size"],
                                      drop_last=False,
                                      shuffle=False,
                                      collate_fn=collate_fn_custom)


485
696
epoch : 0: train loss: 7.182862227925887 Smoothed R interaction (validation) None
epoch : 1: train loss: 6.423080921173096 Smoothed R interaction (validation) None
epoch : 2: train loss: 5.218796342611313 Smoothed R interaction (validation) None
epoch : 3: train loss: 4.578891601700049 Smoothed R interaction (validation) None
epoch : 4: train loss: 4.445278877249131 Smoothed R interaction (validation) None
epoch : 5: train loss: 3.8459851352068095 Smoothed R interaction (validation) None
epoch : 6: train loss: 4.035030769614073 Smoothed R interaction (validation) None
epoch : 7: train loss: 3.886549356465156 Smoothed R interaction (validation) None
epoch : 8: train loss: 3.4298307184989634 Smoothed R interaction (validation) None
epoch : 9: train loss: 3.2201502185601454 Smoothed R interaction (validation) None
epoch : 10: train loss: 3.3437313987658572 Smoothed R interaction (validation) None
epoch : 11: train loss: 3.340399317443371 Smoothed R interaction (validation) None
ep

In [32]:
final_metrics = scripts.evaluate_step(model, test_dataloader, metrics, device)
print(final_metrics)

/home/karolyi/PROJECT_repo/scripts/models.py:56: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1730580467430/work/aten/src/ATen/native/TensorShape.cpp:3683.)
  return torch.linalg.solve(A, Xy).T


{'MSE': 1.8528733253479004, 'R_cellwise': 0.31360891461372375, 'R_cellwise_residuals': 0.25742870569229126}
